# **Libraries**
***
Import needed common libraries

In [8]:
import pandas as pd 
import numpy as np 
import sqlite3

import os
print(os.listdir("../BD00-Datasets"))

['csv-datasets', 'db-datasets', 'New folder', '__MACOSX']


# **Data retrieval**
***
Construct helper function for data connection and queries

In [9]:
#conn = sqlite3.connect('dataset/Northwind_large.sqlite')
#c = conn.cursor()
def run_query(q):
    with sqlite3.connect("../BD00-Datasets/db-datasets/Northwind.sqlite") as conn:
        return pd.read_sql_query(q, conn)

# **Data inspections**
***

View data structures

Construct needed data into dataset

Convert data types

In [10]:
# Inspecting data tables
#run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables = run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables

,name,type
0,Employee,table
1,Category,table
2,Customer,table
3,Shipper,table
4,Supplier,table
5,Order,table
6,Product,table
7,OrderDetail,table
8,CustomerCustomerDemo,table
9,CustomerDemographic,table


In [11]:
q = '''SELECT od.OrderId, od.ProductId, p.ProductName, od.Quantity, od.UnitPrice, 
              (od.Quantity * od.UnitPrice) AS OrderValue, od.Discount,
              ((od.Quantity * od.UnitPrice) - ((od.Quantity * od.UnitPrice) * od.Discount)) AS TotalValue,
              o.OrderDate, o.CustomerId, c.ContactName AS CustomerName,
              o.ShipCountry AS Country 
       FROM OrderDetail AS od
       JOIN Product AS p ON od.ProductId = p.Id
       JOIN 'Order' AS o ON od.OrderId = o.Id
       JOIN Customer AS c ON o.CustomerId = c.Id
    '''

orders = run_query(q)
orders

,OrderId,ProductId,ProductName,Quantity,UnitPrice,OrderValue,Discount,TotalValue,OrderDate,CustomerId,CustomerName,Country
0,10248,11,Queso Cabrales,12,14.00,168.00,0.0,168.00,2012-07-04,VINET,Paul Henriot,France
1,10248,42,Singaporean Hokkien Fried Mee,10,9.80,98.00,0.0,98.00,2012-07-04,VINET,Paul Henriot,France
2,10248,72,Mozzarella di Giovanni,5,34.80,174.00,0.0,174.00,2012-07-04,VINET,Paul Henriot,France
3,10249,14,Tofu,9,18.60,167.40,0.0,167.40,2012-07-05,TOMSP,Karin Josephs,Germany
4,10249,51,Manjimup Dried Apples,40,42.40,1696.00,0.0,1696.00,2012-07-05,TOMSP,Karin Josephs,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...
621801,27065,22,Gustaf's Knäckebröd,20,21.00,420.00,0.0,420.00,2013-08-31 02:59:28,ANATR,Ana Trujillo,Venezuela
621802,27065,77,Original Frankfurter grüne Soße,11,13.00,143.00,0.0,143.00,2013-08-31 02:59:28,ANATR,Ana Trujillo,Venezuela
621803,27065,17,Alice Mutton,45,39.00,1755.00,0.0,1755.00,2013-08-31 02:59:28,ANATR,Ana Trujillo,Venezuela
621804,27065,6,Grandma's Boysenberry Spread,7,25.00,175.00,0.0,175.00,2013-08-31 02:59:28,ANATR,Ana Trujillo,Venezuela


In [12]:
# Import datetime library
from datetime import datetime, timedelta

In [13]:
orders["OrderDate"] = pd.to_datetime(orders["OrderDate"])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621806 entries, 0 to 621805
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   OrderId       621806 non-null  int64         
 1   ProductId     621806 non-null  int64         
 2   ProductName   621806 non-null  object        
 3   Quantity      621806 non-null  int64         
 4   UnitPrice     621806 non-null  float64       
 5   OrderValue    621806 non-null  float64       
 6   Discount      621806 non-null  float64       
 7   TotalValue    621806 non-null  float64       
 8   OrderDate     621806 non-null  datetime64[ns]
 9   CustomerId    621806 non-null  object        
 10  CustomerName  621806 non-null  object        
 11  Country       621806 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 56.9+ MB


# **Aggregate data at customer level**
***

Recency: today date - last order date by each customer 

Frequency: the count of unique order/invoice by each customer

Monetary: the total amount (sum) of paid expenses by each customer

In [14]:
today = orders['OrderDate'].max().date() + timedelta(days=5)
#df_customer = orders.groupby('CustomerId').agg({'OrderDate': lambda x: (today - x.max().date()).days,                          #                                                  'OrderId': lambda x: len(x.unique()),  
#                                               'TotalValue': lambda x: round(x.sum(),2)})
#df_customer.columns = ['Recency', 'Frequency', 'Monetary']
#df_customer.sort_values('Recency', ascending=False)

df_customer = pd.DataFrame(orders.pivot_table(
                           index="CustomerId", 
                           values=["OrderDate", "OrderId", "TotalValue"], 
                           aggfunc={"OrderDate": lambda x: (today - x.max().date()).days,
                                      "OrderId": pd.Series.nunique, 
                                   "TotalValue": lambda x: round(x.sum(),2)}))

df_customer.columns = ["Recency", "Frequency", "Monetary"]
df_customer.sort_values('Recency', ascending=False)

,Recency,Frequency,Monetary
CustomerId,,,
RICSU,45,195,5094316.46
SAVEA,33,218,5394854.75
WOLZA,31,187,5196525.31
BSBEV,30,193,4908283.21
HUNGC,29,197,5410736.54
...,...,...,...
VICTE,5,182,4725721.66
VINET,5,191,5584238.62
ROMEY,5,185,5265845.64


# **Recency, Frequency, and Monetary score**
***

Segregate Recency, Frequency, and Monetary values into discrete values
and classify them into categorical order

In [15]:
def RFM_Quartile (df, rfm, rev=False):
    if (rev == True):
      lbl = [4, 3, 2, 1]
    else: lbl = [1, 2, 3, 4]

    return (pd.cut(df[rfm], 
                   bins=[-1, np.percentile(df[rfm], 25), 
                             np.percentile(df[rfm], 50), 
                             np.percentile(df[rfm], 75), 
                             df[rfm].max()], 
                   labels=lbl).astype("int"))      

In [16]:
# Recency score: 
# Days since last purchase: 1 (17++ days) || 2 (11-16 days) || 3 (8-10 days) || 4 (within a week)
df_customer['R_Score'] = RFM_Quartile(df_customer, "Recency", True )
                                    
# Frequency score: 
# Total number of orders: 1 (177--) || 2 (178-183) || 3 (184-193) || 4 (194++)
df_customer["F_Score"] = RFM_Quartile(df_customer, "Frequency")

# Monetary score: 
# Total paid expenses: 1 (4761k--) || 2 (4782k-5055k) || 3 (5069k-5269k) || 4 (5277k++)
df_customer["M_Score"] = RFM_Quartile(df_customer, "Monetary")

df_customer.head().sort_values('Monetary', ascending=False)

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score
CustomerId,,,,,,
BERGS,6,206,5675614.51,4,4,4
ALFKI,7,200,5317193.08,4,4,4
ANTON,6,191,5262405.09,4,3,3
ANATR,7,192,5208686.42,4,3,3
AROUT,15,162,4395220.43,2,1,1


# **RFM score**
***

The combined (concatenated) score of Recency, Frequency, and Monetary scores

In [17]:
df_customer["RFM_Score"] = ((df_customer["R_Score"].map(str)) + 
                            (df_customer["F_Score"].map(str)) + 
                             df_customer["M_Score"].map(str))

df_customer.head().sort_values('Monetary', ascending=False)

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
CustomerId,,,,,,,
BERGS,6,206,5675614.51,4,4,4,444
ALFKI,7,200,5317193.08,4,4,4,444
ANTON,6,191,5262405.09,4,3,3,433
ANATR,7,192,5208686.42,4,3,3,433
AROUT,15,162,4395220.43,2,1,1,211


# **Segment Labeling**
***

In [18]:
conditions = [
# "Premium Customers" ----------------------------------------------
            (df_customer["RFM_Score"] == '444'), 
# "Loyal Customers" ------------------------------------------------
            (df_customer["RFM_Score"] == '443') 
                | (df_customer["RFM_Score"] == '434') 
                    | (df_customer["RFM_Score"] == '433') |
            (df_customer["RFM_Score"] == '344') 
                | (df_customer["RFM_Score"] == '343') 
                    | (df_customer["RFM_Score"] == '334'), 
# "Frequent Buyers" -----------------------------------------------
            (df_customer["RFM_Score"] == '442') 
                | (df_customer["RFM_Score"] == '441')
                    | (df_customer["RFM_Score"] == '342') 
                        | (df_customer["RFM_Score"] == '341') | 
            (df_customer["RFM_Score"] == '432') 
                | (df_customer["RFM_Score"] == '431')
                    | (df_customer["RFM_Score"] == '333')
                        | (df_customer["RFM_Score"] == '332') 
                            | (df_customer["RFM_Score"] == '331'),
# "Big Spenders" --------------------------------------------------
            (df_customer["RFM_Score"] == '424') 
                | (df_customer["RFM_Score"] == '414') 
                    | (df_customer["RFM_Score"] == '324') 
                        | (df_customer["RFM_Score"] == '314') |
            (df_customer["RFM_Score"] == '423') 
                | (df_customer["RFM_Score"] == '413') 
                    | (df_customer["RFM_Score"] == '323') 
                        | (df_customer["RFM_Score"] == '313'), 
# "Promising New Customers" ---------------------------------------
            (df_customer["RFM_Score"] == '422') 
                | (df_customer["RFM_Score"] == '421')
                    | (df_customer["RFM_Score"] == '412') 
                        | (df_customer["RFM_Score"] == '411') |
            (df_customer["RFM_Score"] == '322') 
                | (df_customer["RFM_Score"] == '321')
                    | (df_customer["RFM_Score"] == '312') 
                        | (df_customer["RFM_Score"] == '311'),
# "Inactive Frequent Buyers" --------------------------------------
            (df_customer["RFM_Score"] == '244') 
                | (df_customer["RFM_Score"] == '243') 
                    | (df_customer["RFM_Score"] == '242') 
                        | (df_customer["RFM_Score"] == '241') |
            (df_customer["RFM_Score"] == '144') 
                | (df_customer["RFM_Score"] == '143') 
                    | (df_customer["RFM_Score"] == '142') 
                        | (df_customer["RFM_Score"] == '141') |
            (df_customer["RFM_Score"] == '232') 
                | (df_customer["RFM_Score"] == '231')
                    | (df_customer["RFM_Score"] == '132') 
                        | (df_customer["RFM_Score"] == '131'),
# "Inactive High Value" -------------------------------------------
            (df_customer["RFM_Score"] == '234') 
                | (df_customer["RFM_Score"] == '224') 
                    | (df_customer["RFM_Score"] == '214') 
                        | (df_customer["RFM_Score"] == '134') 
                            | (df_customer["RFM_Score"] == '124') |
            (df_customer["RFM_Score"] == '233')
                | (df_customer["RFM_Score"] == '223') 
                    | (df_customer["RFM_Score"] == '213')
                        | (df_customer["RFM_Score"] == '133')
                            | (df_customer["RFM_Score"] == '123'), 
# "Inactive Customers" --------------------------------------------
            (df_customer["RFM_Score"] == '222') 
                | (df_customer["RFM_Score"] == '221')
                    | (df_customer["RFM_Score"] == '212')
                        | (df_customer["RFM_Score"] == '211'),
# "Lost Customers" ------------------------------------------------
            (df_customer["RFM_Score"] == '122') 
                | (df_customer["RFM_Score"] == '121')
                    | (df_customer["RFM_Score"] == '114') 
                        | (df_customer["RFM_Score"] == '113') 
                            | (df_customer["RFM_Score"] == '112')
# "Lost Cheap Customers" ------------------------------------------
#                | (df_customer["RFM_Score"] == 111)
]

In [19]:
mappings = ["Premium Customers", 
            "Loyal Customers", 
            "Frequent Buyers", 
            "Big Spenders", 
            "Promising New Customer", 
            "Inactive Frequent Buyers",
            "Inactive High Value",
            "Inactive Customers",
            "Lost Customers"]

df_customer["RFM_Segment"] = np.select(conditions, mappings, default="Lost Cheap Customers")
df_customer["RFM_Segment"].value_counts()

Promising New Customer      15
Loyal Customers             13
Inactive High Value         11
Inactive Frequent Buyers    11
Inactive Customers           9
Lost Cheap Customers         7
Premium Customers            7
Frequent Buyers              7
Lost Customers               6
Big Spenders                 5
Name: RFM_Segment, dtype: int64

In [20]:
promotions = ["New Product & Loyalty programs", 
              "Priced Bundling & Loyalty programs",
              "Mixed Bundling & Discount Sale",
              "New Product & Priced Bundling",
              "Point Rewards & Discount Sale",
              "Returning Offers & Discount Sale",
              "Returning Offers & New Product",
              "Login Offers & Agressive Discount",
              "Nothing much but Login Offers"]

df_customer["Promotion"] = np.select(conditions, promotions, default="Don't bother to reaquire")
df_customer["Promotion"].value_counts()

Point Rewards & Discount Sale         15
Priced Bundling & Loyalty programs    13
Returning Offers & Discount Sale      11
Returning Offers & New Product        11
Login Offers & Agressive Discount      9
New Product & Loyalty programs         7
Mixed Bundling & Discount Sale         7
Don't bother to reaquire               7
Nothing much but Login Offers          6
New Product & Priced Bundling          5
Name: Promotion, dtype: int64

In [21]:
df_customer

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,RFM_Segment,Promotion
CustomerId,,,,,,,,,
ALFKI,7,200,5317193.08,4,4,4,444,Premium Customers,New Product & Loyalty programs
ANATR,7,192,5208686.42,4,3,3,433,Loyal Customers,Priced Bundling & Loyalty programs
ANTON,6,191,5262405.09,4,3,3,433,Loyal Customers,Priced Bundling & Loyalty programs
AROUT,15,162,4395220.43,2,1,1,211,Inactive Customers,Login Offers & Agressive Discount
BERGS,6,206,5675614.51,4,4,4,444,Premium Customers,New Product & Loyalty programs
...,...,...,...,...,...,...,...,...,...
WARTH,9,192,5241505.28,3,3,3,333,Frequent Buyers,Mixed Bundling & Discount Sale
WELLI,11,185,5280365.28,2,3,4,234,Inactive High Value,Returning Offers & New Product
WHITC,16,170,4679821.89,2,1,1,211,Inactive Customers,Login Offers & Agressive Discount
